In [ ]:
import pandas as pd
from geopy.distance import geodesic
from pandarallel import pandarallel
import gmaps

In [ ]:
pandarallel.initialize(nb_workers=10)

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train = df_train.reset_index(drop=False)
df_test = df_test.reset_index(drop=False)

# Integrate auxiliary data

In [ ]:
df_mrt_existing = pd.read_csv('auxiliary-data/sg-mrt-existing-stations.csv')
df_mrt_planned = pd.read_csv('auxiliary-data/sg-mrt-planned-stations.csv')
df_primary_school = pd.read_csv('auxiliary-data/sg-primary-schools.csv')
df_shopping_mall = pd.read_csv('auxiliary-data/sg-shopping-malls.csv')

df_mrt_existing = df_mrt_existing[['name','latitude','longitude']]
df_mrt_planned = df_mrt_planned[['name','latitude','longitude']]

In [ ]:
print('number of existing mrt stations: ', len(df_mrt_existing))
print('number of planned mrt stations: ', len(df_mrt_planned))
print('number of primary schools: ', len(df_primary_school))
print('number of shopping malls: ', len(df_shopping_mall))

In [ ]:
df_mrt_existing.isnull().sum()

In [ ]:
df_mrt_planned.isnull().sum()

In [ ]:
df_primary_school.isnull().sum()

In [ ]:
df_shopping_mall.isnull().sum()

In [ ]:
def find_min_and_second_smallest(group):
    min_value = group.min()
    second_min_value = group.loc[group > min_value].min()
    return min_value, second_min_value

In [ ]:
def nearest_mrt(df_base, df_mrt):
    df_base['key'] = 0
    df_mrt['key'] = 0
    df_merge = pd.merge(df_base, df_mrt, on='key', how='outer')
    df_merge['distance'] = df_merge.parallel_apply(lambda row: geodesic((row['latitude_x'],row['longitude_x']), (row['latitude_y'],row['longitude_y'])).m, axis=1)
    df_merge = df_merge[['index','distance']]
    df_nearest = df_merge.groupby('index')['distance'].apply(find_min_and_second_smallest)
    df_nearest = df_nearest.reset_index()
    df_nearest[['first_mrt_distance', 'second_mrt_distance']] = df_nearest['distance'].apply(lambda x: pd.Series(x))
    df_first = pd.merge(df_merge, df_nearest[['index','first_mrt_distance']], left_on=['index','distance'], right_on=['index','first_mrt_distance'], how='inner')
    df_second = pd.merge(df_merge, df_nearest[['index','second_mrt_distance']], left_on=['index','distance'], right_on=['index','second_mrt_distance'], how='inner')
    df_first = df_first.groupby(['index', 'distance']).count().reset_index()
    df_second = df_second.groupby(['index', 'distance']).count().reset_index()
    df_result = pd.merge(df_first, df_second, on='index', how='inner')
    df_result = df_result.rename(columns={'distance_x':'first_mrt_distance', 'first_mrt_distance':'first_mrt_lines', 'distance_y':'second_mrt_distance', 'second_mrt_distance':'second_mrt_lines'})
    return df_result

In [ ]:
def nearest_amenity(df_base, df_amenity):
    df_base['key'] = 0
    df_amenity['key'] = 0
    df_merge = pd.merge(df_base, df_amenity, on='key', how='outer')
    df_merge['distance'] = df_merge.parallel_apply(lambda row: geodesic((row['latitude_x'],row['longitude_x']), (row['latitude_y'],row['longitude_y'])).m, axis=1)
    df_merge = df_merge[['index','name','distance']]
    df_nearest = df_merge.groupby('index')['distance'].apply(find_min_and_second_smallest)
    df_nearest = df_nearest.reset_index()
    df_nearest[['first_amenity_distance', 'second_amenity_distance']] = df_nearest['distance'].apply(lambda x: pd.Series(x))
    df_first = pd.merge(df_merge, df_nearest[['index','first_amenity_distance']], left_on=['index','distance'], right_on=['index','first_amenity_distance'], how='inner')
    df_second = pd.merge(df_merge, df_nearest[['index','second_amenity_distance']], left_on=['index','distance'], right_on=['index','second_amenity_distance'], how='inner')
    df_result = pd.merge(df_first, df_second, on='index', how='inner')
    df_result = df_result.rename(columns={'name_x':'first_amenity_name', 'name_y':'second_amenity_name'})
    df_result = df_result[['index','first_amenity_name','first_amenity_distance','second_amenity_name','second_amenity_distance']]
    return df_result

In [ ]:
def integrate(df, df_mrt_existing, df_mrt_planned, df_primary_school, df_shopping_mall):
    df_base = df[['index','latitude','longitude']]

    nearest_existing_mrt = nearest_mrt(df_base.copy(), df_mrt_existing.copy())
    nearest_planned_mrt = nearest_mrt(df_base.copy(), df_mrt_planned.copy())
    nearest_primary_school = nearest_amenity(df_base.copy(), df_primary_school.copy())
    nearest_shopping_mall = nearest_amenity(df_base.copy(), df_shopping_mall.copy())

    nearest_existing_mrt = nearest_existing_mrt.rename(columns={'first_mrt_distance':'first_existing_mrt_distance', 'first_mrt_lines':'first_existing_mrt_lines',
                                                                'second_mrt_distance':'second_existing_mrt_distance', 'second_mrt_lines':'second_existing_mrt_lines'})
    nearest_planned_mrt = nearest_planned_mrt.rename(columns={'first_mrt_distance':'first_planned_mrt_distance', 'first_mrt_lines':'first_planned_mrt_lines',
                                                                'second_mrt_distance':'second_planned_mrt_distance', 'second_mrt_lines':'second_planned_mrt_lines'})
    nearest_primary_school = nearest_primary_school.rename(columns={'first_amenity_name':'first_school_name', 'first_amenity_distance':'first_school_distance',
                                                                    'second_amenity_name':'second_school_name', 'second_amenity_distance':'second_school_distance'})
    nearest_shopping_mall = nearest_shopping_mall.rename(columns={'first_amenity_name':'first_mall_name', 'first_amenity_distance':'first_mall_distance',
                                                                    'second_amenity_name':'second_mall_name', 'second_amenity_distance':'second_mall_distance'})

    df = pd.merge(df, nearest_existing_mrt, on='index', how='inner')
    df = pd.merge(df, nearest_planned_mrt, on='index', how='inner')
    df = pd.merge(df, nearest_primary_school, on='index', how='inner')
    df = pd.merge(df, nearest_shopping_mall, on='index', how='inner')

    return df

In [ ]:
df_train = integrate(df_train.copy(), df_mrt_existing, df_mrt_planned, df_primary_school, df_shopping_mall)
df_test = integrate(df_test.copy(), df_mrt_existing, df_mrt_planned, df_primary_school, df_shopping_mall)

df_train.to_csv('integrated_train.csv', index=False)
df_test.to_csv('integrated_test.csv', index=False)

# Generate heat map

In [18]:
gmaps.configure(api_key='')

In [19]:
locations = df_train[['latitude', 'longitude']]
price = df_train['monthly_rent']

In [20]:
fig = gmaps.figure() 
heatmap_layer = gmaps.heatmap_layer(locations, weights=price) 
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))